In [1]:
# file_path = "data/All/Pos_all_mature_seq.csv"
# file_path2 = "data/All/Neg_all_mature_seq.csv"

In [1]:
import pandas as pd
import numpy as np
import xlrd
import math
from Fn import Fn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from CNN import CNN
import torch
import os
from torch.utils.data import DataLoader,TensorDataset
import utils.metricsCal

In [2]:
# pos_data=pd.read_csv(file_path, header=None)
# neg_data=pd.read_csv(file_path2,header=None)
# pos_data

In [3]:
import random

random.seed(1)


def fileTranfor(data, flag, save_path, num_samples=None):
    rows = list(range(len(data)))
    #rows = list(range(data.shape[0]))
    if not flag:
        print(num_samples)
        rows = random.sample(rows, num_samples)

    with open(save_path, 'w') as f:
        for i, row in enumerate(rows):
            if flag:
                f.write('>P' + str(i + 1) + '\n')
            else:
                f.write('>N' + str(i + 1) + '\n')
            f.write((data.iloc[row][:].values)[0][480:521] + '\n')

In [4]:
# fileTranfor(neg_data, False, 'data/All/neg.fasta', len(pos_data))

In [5]:
# fileTranfor(pos_data, True, 'data/All/pos.fasta', len(pos_data))

In [2]:
file_path3 = "all_pos_result.xls"
file_path4 = "all_neg_result.xls"

In [7]:
# fileHandler = open("all_pos_result.xls", "r")
# p = 0
# while True:
#     p += 1
#     # Get next line from file
#     line = fileHandler.readline()
#     # If line is empty then end of file reached
#     if not line:
#         break
#     #print(line.strip())
#     # Close Close
# fileHandler.close()

In [8]:
# p

In [9]:
# fileHandler = open("all_neg_result.xls", "r")
# n = 0
# while True:
#     n += 1
#     # Get next line from file
#     line = fileHandler.readline()
#     # If line is empty then end of file reached
#     if not line:
#         break
#     #print(line.strip())
#     # Close Close
# fileHandler.close()

In [10]:
# n

In [11]:
# fileHandler = open("all_pos_result.xls", "r")
# flag1 = False
# n = 0
# mean = ""
# while True:
#     n += 1
#     # Get next line from file
#     line = fileHandler.readline()
#     if flag1 and line.strip() != "Score     E":
#         k = 0
#         print(n)
#         print(mean)
#     flag1 = False

#     if line.strip().find("Query=") != -1:
#         line1 = fileHandler.readline()
#         line2 = fileHandler.readline()
#         mean = line.strip()
#         flag1 = True
#     if not line:
#         break
#     # Close Close
# fileHandler.close()

In [12]:
# fileHandler = open("all_neg_result.xls", "r")
# n = 0
# while True:
#     n += 1
#     line = fileHandler.readline()
#     if n == 16:
#         print(line.strip())
#         if line.strip().find("Query=") != -1:
#             print("saasd")
#         #print(line.strip()=="Score     E")
#         if line.strip() == "Score     E":
#             print("sad")
#         #line.strip()=="Score     E"
#     if not line:
#         break
#     # Close Close
# fileHandler.close()

In [3]:
pos_id = []
pos_fea = []
fileHandler = open("all_pos_result.xls", "r")
while True:
    line = fileHandler.readline()
    if line.strip().find("Query=") != -1:
        pos_id.append(line.strip())
        fea = []
        line = fileHandler.readline()
        line = fileHandler.readline()
        line = fileHandler.readline()
        if line.strip().find("Score     E") != -1:
            line = fileHandler.readline()
            line = fileHandler.readline()
            line = fileHandler.readline()
            while line.strip() != "":
                fea.append(line.strip())
                line = fileHandler.readline()
        pos_fea.append(fea)
    if not line:
        break
fileHandler.close()

In [4]:
len(pos_id)

3430

In [5]:
len(pos_fea)

3430

In [6]:
neg_id = []
neg_fea = []
fileHandler = open("all_neg_result.xls", "r")
while True:
    line = fileHandler.readline()
    if line.strip().find("Query=") != -1:
        neg_id.append(line.strip())
        fea = []
        line = fileHandler.readline()
        line = fileHandler.readline()
        line = fileHandler.readline()
        if line.strip().find("Score     E") != -1:
            line = fileHandler.readline()
            line = fileHandler.readline()
            line = fileHandler.readline()
            while line.strip() != "":
                fea.append(line.strip())
                line = fileHandler.readline()
        neg_fea.append(fea)
    if not line:
        break
fileHandler.close()

In [7]:
len(neg_id)

3430

In [8]:
len(neg_fea)

3430

In [9]:
neg_fea[0]

['P12429                                                              28.3    0.047',
 'P12031                                                              28.3    0.047',
 'P4473                                                               24.7    0.57',
 'P4150                                                               24.7    0.57',
 'N6236                                                               22.9    2.0',
 'P11562                                                              22.9    2.0',
 'P3609                                                               22.9    2.0',
 'P3608                                                               22.9    2.0',
 'P2905                                                               22.9    2.0',
 'N12517                                                              21.1    6.9',
 'N11730                                                              21.1    6.9',
 'N9925                                                               

In [10]:
#检验数量合理性
p = 0
n = 0
for i in range(len(pos_fea)):
    if len(pos_fea[i]) == 0:
        p += 1
    if len(neg_fea[i]) == 0:
        n += 1

In [11]:
neg_fea[0][0].split()[0][0]

'P'

In [12]:
neg_fea[0][0].split()[1]

'28.3'

In [13]:
neg_fea[0][0].split()

['P12429', '28.3', '0.047']

In [14]:
neg_fea[0][0].split()[0][0]

'P'

In [15]:
neg_predict = []
pos_predict = []
for i in range(len(neg_fea)):
    sum_ = 0
    pos = 0
    for j in range(len(neg_fea[i])):
        if neg_fea[i][j].split()[0][0]=="P":
            sum_+=float(neg_fea[i][j].split()[1])
            pos+=float(neg_fea[i][j].split()[1])
        else:
            sum_+=float(neg_fea[i][j].split()[1])
    if sum_!=0:
        neg_predict.append(pos/sum_)
    else:
        neg_predict.append(0.5)

In [16]:
len(neg_predict)

3430

In [17]:
for i in range(len(pos_fea)):
    sum_ = 0
    pos = 0
    for j in range(len(pos_fea[i])):
        if pos_fea[i][j].split()[0][0]=="P":
            sum_+=float(pos_fea[i][j].split()[1])
            pos+=float(pos_fea[i][j].split()[1])
        else:
            sum_+=float(pos_fea[i][j].split()[1])
    if sum_!=0:
        pos_predict.append(pos/sum_)
    else:
        pos_predict.append(0.5)

In [18]:
len(pos_predict)

3430

In [19]:
y_pred = np.append(pos_predict, neg_predict)
posLabel = np.ones(len(pos_predict))
negLabel = np.zeros(len(neg_predict))
y_true = np.append(posLabel,negLabel)

In [47]:
mcc_threshold,TN,FN,FP,TP,Sen,Spe,Acc,max_mcc,AUC = get_train_metrics(y_pred,y_true)
print("Acc:",Acc)
print("max_mcc:",max_mcc)
print("AUC:",AUC)
mcc_threshold

Acc: 0.6036443148688047
max_mcc: 0.20803868878755694
AUC: 0.6346593256211273


0.5020708697653011

In [20]:
def get_train_metrics(y_pred, y_true):  #获取训练指标
    y_pred = np.array(y_pred)  #预测值，转化为numpy
    y_true = np.array(y_true)  #准确值，转化为numpy
    ## [::-1]反序
    desc_score_indices = np.argsort(y_pred, kind="mergesort")[::-1]  #使用归并排序，让y_pred降序，返回标签序
    y_pred = y_pred[desc_score_indices]
    y_true = y_true[desc_score_indices]

    TP = FP = 0
    TN = np.sum(y_true==0)  #结果为0的数量
    FN = np.sum(y_true==1)   #结果为1的数量
    mcc = 0
    mcc_threshold = y_pred[0]+1  #预测的最大值加1
    confuse_matrix = (TP,FP,TN,FN)  #定义混淆矩阵
    max_mcc = -1  #定义最大mcc值用于记录
    ## score 降序排序， y >= score 前半部分预测为正
    for index,score in enumerate(y_pred):  #取索引+预测值
        if y_true[index] == 1:
            TP += 1
            FN -= 1
        else:
            FP += 1
            TN -= 1
        numerator = (TP*TN - FP*FN)
        denominator = (math.sqrt(TP+FP) * math.sqrt(TN+FN) * math.sqrt(TP+FN) * math.sqrt(TN+FP))
        if denominator == 0:
            mcc = 0
        else:
            mcc = numerator / denominator
        
        if mcc > max_mcc:
            max_mcc = mcc
            confuse_matrix = (TP, FP, TN, FN)
            mcc_threshold = score  #找出最好的阈值
    TP, FP, TN, FN = confuse_matrix  #混淆矩阵得出结果
    Sen = 0 if (TP + FN) == 0 else (TP / (TP + FN))  #计算……
    Spe = 0 if (TN + FP) == 0 else (TN / (TN + FP))
    Acc = 0 if (TP + FP + TN + FN) == 0 else ((TP + TN) / (TP + FP + TN + FN))
    AUC = roc_auc_score(y_true, y_pred)
    return mcc_threshold,TN,FN,FP,TP,Sen,Spe,Acc,max_mcc,AUC  #返回一系列结果

In [32]:
for i in range(len(neg_fea)):
    sum_ = 0
    pos = 0
    for j in range(len(neg_fea[i])):
        if neg_fea[i][j].split()[0][0]=="P":
            sum_+=float(neg_fea[i][j].split()[1])
            pos+=float(neg_fea[i][j].split()[1])
        else:
            sum_+=float(neg_fea[i][j].split()[1])
    if sum_!=0:
        neg_predict.append(pos/sum_)
    else:
        neg_predict.append(0.5)
for i in range(len(pos_fea)):
    sum_ = 0
    pos = 0
    for j in range(len(pos_fea[i])):
        if pos_fea[i][j].split()[0][0]=="P":
            sum_+=float(pos_fea[i][j].split()[1])
            pos+=float(pos_fea[i][j].split()[1])
        else:
            sum_+=float(pos_fea[i][j].split()[1])
    if sum_!=0:
        pos_predict.append(pos/sum_)
    else:
        pos_predict.append(0.5)

In [33]:
###############通过神经网络计算评分##################

In [21]:
pos_feature = np.zeros(len(neg_fea) * 150).reshape(len(neg_fea), 150)
neg_feature = np.zeros(len(neg_fea) * 150).reshape(len(neg_fea), 150)
max_abs = 0
min_abs= 10000
###阳性获取特征结果
for i in range(len(pos_fea)):
    for j in range(len(pos_fea[i])):
        if float(pos_fea[i][j].split()[1])>max_abs:
            max_abs = float(pos_fea[i][j].split()[1])
        if float(pos_fea[i][j].split()[1])<min_abs:
            min_abs = float(pos_fea[i][j].split()[1])
        if pos_fea[i][j].split()[0][0]=="P":
            pos_feature[i][j] = float(pos_fea[i][j].split()[1])
        else:
            pos_feature[i][j] = float(pos_fea[i][j].split()[1])*(-1)
        
for i in range(len(neg_fea)):
    for j in range(len(neg_fea[i])):
        if float(neg_fea[i][j].split()[1])>max_abs:
            max_abs = float(neg_fea[i][j].split()[1])
        if float(neg_fea[i][j].split()[1])<min_abs:
            min_abs = float(neg_fea[i][j].split()[1])
        if neg_fea[i][j].split()[0][0]=="P":
            neg_feature[i][j] = float(neg_fea[i][j].split()[1])
        else:
            neg_feature[i][j] = float(neg_fea[i][j].split()[1])*(-1)

In [22]:
min_abs

21.1

In [23]:
for i in range(len(pos_feature)):
    for j in range(len(pos_feature[i])):
        if pos_feature[i][j]>0:
            pos_feature[i][j] = (pos_feature[i][j]-min_abs-1)/(max_abs-min_abs)
        elif pos_feature[i][j]<0:
            pos_feature[i][j] = (pos_feature[i][j]+min_abs-1)/(max_abs-min_abs)
for i in range(len(neg_feature)):
    for j in range(len(neg_feature[i])):
        if neg_feature[i][j]>0:
            neg_feature[i][j] = (neg_feature[i][j]-min_abs-1)/(max_abs-min_abs)
        elif neg_feature[i][j]<0:
            neg_feature[i][j] = (neg_feature[i][j]+min_abs-1)/(max_abs-min_abs)

In [24]:
trainData = np.append(pos_feature,neg_feature,axis=0)

In [25]:
y_true.shape

(6860,)

In [26]:
###制造Label，然后合并数据，使用KFold和SVM查看结果###
y_true
kf = KFold(5, True, 10)

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [28]:
AUC=[]
for i,[train_index, test_index] in enumerate(kf.split(trainData)):
    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = y_true[train_index]
    Y_test = y_true[test_index]
    batch_size = 128
    best_auc = 0
    patience = 0
    max_epochs = 200
    model_dir = 'model/FN_Blastn/KFold_' + str(i) + '/'
    #model = CNN.CNN()
    model = Fn.Fn()
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  # 选择设备
    model = model.to(device)
    os.environ['CUDA_VISIBLE_DEVICES'] = '1'
    for j in range(max_epochs):
        CNN.train(model,X_train,Y_train,i,model_dir,batch_size)
        auc=test(X_test,Y_test,best_auc,model_dir,batch_size,0.5)
        if auc > best_auc:
            best_auc=auc
            patience=0
        else:
            patience+=1
        if patience==10:
            break
    AUC.append(best_auc)

29.66415375471115
Accuracy on test set: 52 %
Sensitivity on test set: 87 %
Speciality on test set: 18 %
auc on test set: 0.593
The number of correct: 723
29.448366940021515
Accuracy on test set: 55 %
Sensitivity on test set: 74 %
Speciality on test set: 35 %
auc on test set: 0.600
The number of correct: 758
29.499759256839752
Accuracy on test set: 55 %
Sensitivity on test set: 24 %
Speciality on test set: 87 %
auc on test set: 0.599
The number of correct: 764
29.316757380962372
Accuracy on test set: 51 %
Sensitivity on test set: 94 %
Speciality on test set: 8 %
auc on test set: 0.595
The number of correct: 706
29.431311666965485
Accuracy on test set: 55 %
Sensitivity on test set: 60 %
Speciality on test set: 51 %
auc on test set: 0.591
The number of correct: 768
29.338802933692932
Accuracy on test set: 54 %
Sensitivity on test set: 69 %
Speciality on test set: 40 %
auc on test set: 0.600
The number of correct: 754
29.281290590763092
Accuracy on test set: 55 %
Sensitivity on test set: 6

29.650605142116547
Accuracy on test set: 56 %
Sensitivity on test set: 49 %
Speciality on test set: 62 %
auc on test set: 0.596
The number of correct: 770
29.388355672359467
Accuracy on test set: 51 %
Sensitivity on test set: 98 %
Speciality on test set: 6 %
auc on test set: 0.599
The number of correct: 703
29.3637512922287
Accuracy on test set: 56 %
Sensitivity on test set: 60 %
Speciality on test set: 51 %
auc on test set: 0.598
The number of correct: 769
29.382575929164886
Accuracy on test set: 56 %
Sensitivity on test set: 75 %
Speciality on test set: 37 %
auc on test set: 0.598
The number of correct: 771
29.24887627363205
Accuracy on test set: 55 %
Sensitivity on test set: 73 %
Speciality on test set: 38 %
auc on test set: 0.597
The number of correct: 767
29.26781666278839
Accuracy on test set: 53 %
Sensitivity on test set: 87 %
Speciality on test set: 21 %
auc on test set: 0.597
The number of correct: 732
29.426903903484344
Accuracy on test set: 50 %
Sensitivity on test set: 99 %

In [27]:
def test(data,label,best_auc,model_dir,batch_size,th):
    model_test = load_model(model_dir)
    model_test.eval()
    model_test.to(device)
    data = torch.Tensor(data)
    label = torch.Tensor(label)
    test_data = TensorDataset(data,label)
    test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)
    TP,TN,FP,FN=0,0,0,0
    total = 0
    outputs_list = []
    target_list = []
    with torch.no_grad():
        for data in test_loader:
            inputs,target = data
            inputs = inputs.reshape(inputs.shape[0], 1, inputs.shape[1])
            inputs = inputs.to(device)
            target = target.to(device)
            target = target.reshape(target.shape[0], 1)
            outputs = model_test(inputs)
            outputs_list+=outputs.tolist()
            target_list+=target.tolist()

            total += target.size(0)
            for i in range(target.size(0)):
                if target[i]==1:
                    if outputs[i]>=th:
                        TP+=1
                    else:
                        FN+=1
                if target[i] == 0:
                    if outputs[i]>=th:
                        FP+=1
                    else:
                        TN+=1
    outputs_list = torch.Tensor(outputs_list)
    target_list = torch.Tensor(target_list)
    auc_ = utils.metricsCal.auc_cal(outputs_list,target_list)
    print('Accuracy on test set: %d %%' %(100*(TP+TN)/total))
    print('Sensitivity on test set: %d %%' % (100 * TP / (TP+FN)))
    print('Speciality on test set: %d %%' % (100 * TN / (TN+FP)))
    #print('MCC on test set: %.3f' % float((TP*TN-FP*FN) /math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))))
    print('auc on test set: %.3f' %auc_)
    print('The number of correct: %d' %(TP+TN))
    if(auc_ > best_auc):
        torch.save(model_test,model_dir+'model_best.pt')
    return auc_
def load_model(model_dir):
    if os.path.exists(model_dir+'model.pt'):
        model_load = torch.load(model_dir+'model.pt')
    return model_load

In [48]:
#直接求概率
AUC

0.6346593256211273

In [53]:
#CNN简单计算的结果
AUC

[0.6072076685734685,
 0.6084803821569427,
 0.5950662966389146,
 0.6006363928242616,
 0.6118853329933274]

In [29]:
#FN简单计算的结果
AUC

[0.6000720363114009,
 0.6043345403236774,
 0.5913022212298106,
 0.5992160439882885,
 0.6128458498023714]